## 1. Contexte

### Utilisateurs

L'Inspection Générale de l'Administration [(IGA)](https://fr.wikipedia.org/wiki/Inspection_g%C3%A9n%C3%A9rale_de_l%27administration) est chargée de missions d’évaluation des politiques publiques, d’audit des services, d’appui, de conseil et de contrôle pour les membres du Gouvernement et rattachée directement au Ministère de l'Intérieur et des Outres-Mers.

### Outil existant
Un moteur de recherche a été développé en 2020 pour les besoins documentaires de l'IGA, ainsi que pour d'autres sercices du MIOM. Une version ouverte est disponible [ici](https://kubernetes.ridoc.fr). info de connexion : `user1:abcxyz`.

## 2. Données
Cette version ouverte est alimentée avec des documents publiques disponibles sur le site internet du [MIOM](https://www.interieur.gouv.fr/Publications/Rapports-de-l-IGA/Bonnes-Feuilles). 
On dénombre petite centaine de rapports qui traitent des sujets comme l'écologie, l'aménagement du territoire, la sécurité intérieure, l'éducation, etc...

Ce sont des synthèses choisies de rapport bien plus longs et documentés disponibles sur un intranet privé.

Les données bruts sont disponibles à travers un container s3 OVH `bf-iga`, qui contient un sous dossier `pdf` avec les documents .pdf ainsi que le sous dossier `json` avec les contenus des pdf, assortis de méta-données (`author`, `date`, `title`) .

---
## 3. Objectif : Extraction d'acronymes

On se propose d'extraire les acronymes présents dans des documents textuels, en s'appuyant sur les nouvelles techniques de traîtement du language (*large language modele*, LLM).

--> A partir des documents présents dans le container s3 `bf-iga`, proposer une méthode d'extraction des acronymes avec leurs signications et fournir la liste correspondante.

---
## 4. Consignes

1. Ne pas dépasser 4 heures pour réaliser l'objectif.
2. Visez la simplicité plutôt que la complexité, la qualité plutôt que la quantité.
3. Nous avons mis en place une API de LLM que vous pourrez utiliser. (Cf les exemples ci dessous).

### Critères d'évaluation

1. Savoir expliquer ses hypothèses, connaître les limites de son appoche.
2. Qualité des réponses aux questions posées après la restitution.
3. Clarté du code.

## 5. Des exemples d'utilisation de LLM :

In [1]:
from dotenv import load_dotenv

load_dotenv()



In [5]:
import json
import s3fs
import os
import json
from pathlib import Path

# Import langchain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


True

In [6]:
# La base de données dans un stockage s3
container = 'iga-bf'
s3 = s3fs.S3FileSystem(anon=False, client_kwargs={'endpoint_url': os.getenv("AWS_S3_ENDPOINT"),
                                                  "aws_access_key_id": os.getenv("AWS_S3_KEY_ID"),
                                                  "aws_secret_access_key": os.getenv("AWS_S3_SECRET_KEY")
                                                  })
s3.ls(container)
#for json_path in s3.find(f'{container}/json'):
#    print(json_path)

['iga-bf/json', 'iga-bf/pdf']

In [7]:
# Construire des bouts de documents
text_splitter = CharacterTextSplitter(
    separator=". ",
    chunk_size=500, # 1500
    chunk_overlap=100,
    length_function=len,
    keep_separator=False
)

docs_splitted = []
for json_path in s3.find(f'{container}/json') or ('iga-bf/json/BF2015-01-14125 - Mutualisation CT.json',):
    with s3.open(json_path, 'rb',encoding="UTF-8") as f:
        json_doc = json.load(f)
    content = json_doc['content'].replace(u"\uFFFD", "ti").replace("%", "ti").replace("#", "ti").replace(">", "ti").replace("- ", "")
    doc = Document(page_content=content, metadata={'author': json_doc.get('author',''), 'title': json_doc.get('title','')})
    docs_splitted += text_splitter.split_documents([doc])
print(f"\nIl y a {len(docs_splitted)} bouts de documents")

Created a chunk of size 516, which is longer than the specified 500
Created a chunk of size 819, which is longer than the specified 500
Created a chunk of size 721, which is longer than the specified 500
Created a chunk of size 628, which is longer than the specified 500
Created a chunk of size 520, which is longer than the specified 500
Created a chunk of size 540, which is longer than the specified 500
Created a chunk of size 766, which is longer than the specified 500
Created a chunk of size 556, which is longer than the specified 500
Created a chunk of size 570, which is longer than the specified 500
Created a chunk of size 1268, which is longer than the specified 500
Created a chunk of size 552, which is longer than the specified 500
Created a chunk of size 546, which is longer than the specified 500
Created a chunk of size 612, which is longer than the specified 500
Created a chunk of size 614, which is longer than the specified 500
Created a chunk of size 544, which is longer th


Il y a 2096 bouts de documents


In [8]:
llm = ChatOpenAI(temperature=0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f48cc34b800>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f48ce3a1c10>, root_client=<openai.OpenAI object at 0x7f48cddcfe60>, root_async_client=<openai.AsyncOpenAI object at 0x7f48cc3585f0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api-ai.numerique-interieur.com/v1', openai_proxy='')

In [17]:
prompt_template = ChatPromptTemplate.from_template("Invente la signification de l'acronyme suivant : {acronym}")
chain = prompt_template | llm | StrOutputParser()

In [19]:
print(chain.invoke({"acronym":"IGA"}))

**IGA : Institut Global d'Analyse**

L'Institut Global d'Analyse (IGA) est une organisation internationale dédiée à la recherche et à la diffusion des connaissances sur les tendances globales, les enjeux sociétaux et les défis du 21e siècle.  

* **Recherche:** L'IGA mène des études approfondies sur divers sujets tels que l'économie mondiale, la technologie disruptive, les changements climatiques, la sécurité internationale et les migrations.
* **Diffusion des connaissances:** L'IGA publie des rapports, organise des conférences internationales et développe des programmes d'éducation pour partager ses analyses et recommandations avec un large public.
* **Collaboration:** L'IGA collabore avec des universités, des think tanks, des organisations gouvernementales et du secteur privé pour favoriser le dialogue et la coopération internationale sur les questions globales.


J'espère que cette signification vous convient ! 

